In [4]:
!pip install selenium
!pip install pandas
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Setup Selenium WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service("/usr/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=chrome_options)

url = "https://www.zakonyprolidi.cz/obor/trestni-pravo"
driver.get(url)

# Wait for the page to load
time.sleep(3)

# Extract the table
soup = BeautifulSoup(driver.page_source, "html.parser")
table = soup.find_all("table")[4]

# Get column names and add an extra column for full text
world_titles = table.find_all("th")
world_table_titles = [title.text.strip() for title in world_titles] + ["Aktuální znění"]

# Create an empty DataFrame
df = pd.DataFrame(columns=world_table_titles)

# Function to extract data from a table
def extract_table_data():
    global df
    soup = BeautifulSoup(driver.page_source, "html.parser")
    table = soup.find_all("table")[4]
    column_data = table.find_all("tr")

    for row in column_data[1:]:
        row_data = row.find_all("td")
        individual_row_data = [data.text.strip() for data in row_data]

        # Get full text from the law link
        link_element = row.find("a")
        if link_element and "href" in link_element.attrs:
            predpis_url = "https://www.zakonyprolidi.cz" + link_element["href"]
            print(f"Fetching: {predpis_url}")

            driver.get(predpis_url)
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "Frags")))

            law_soup = BeautifulSoup(driver.page_source, "html.parser")
            law_text_element = law_soup.find("div", class_="Frags")

            if law_text_element:
                full_text = law_text_element.get_text(separator=" ").strip()
                individual_row_data.append(full_text)

        # Append full text to row data
        df.loc[len(df)] = individual_row_data if len(individual_row_data) == len(df.columns) else None

        if len(individual_row_data) != len(df.columns):
            print(f"Skipping row (mismatch): {individual_row_data}")

# Extract data from the first page
extract_table_data()

# Close Selenium
driver.quit()

# Display the final DataFrame
print(df)

Fetching: https://www.zakonyprolidi.cz/cs/2009-40
Fetching: https://www.zakonyprolidi.cz/cs/1961-141
Fetching: https://www.zakonyprolidi.cz/cs/2008-253
Fetching: https://www.zakonyprolidi.cz/cs/2001-120
Fetching: https://www.zakonyprolidi.cz/cs/2003-218
Fetching: https://www.zakonyprolidi.cz/cs/2011-418
Fetching: https://www.zakonyprolidi.cz/cs/1961-140
Fetching: https://www.zakonyprolidi.cz/cs/2023-171
Fetching: https://www.zakonyprolidi.cz/cs/1999-169
Fetching: https://www.zakonyprolidi.cz/cs/2013-45
Fetching: https://www.zakonyprolidi.cz/cs/2023-349
Fetching: https://www.zakonyprolidi.cz/cs/1950-86
Fetching: https://www.zakonyprolidi.cz/cs/2006-115
Fetching: https://www.zakonyprolidi.cz/cs/2025-24
Fetching: https://www.zakonyprolidi.cz/cs/1948-231
Fetching: https://www.zakonyprolidi.cz/cs/2024-166
Fetching: https://www.zakonyprolidi.cz/cs/1999-345
Fetching: https://www.zakonyprolidi.cz/cs/2018-67
Fetching: https://www.zakonyprolidi.cz/cs/2000-257
Fetching: https://www.zakonyprolidi.

In [5]:
df.to_csv("czech_laws.csv", index=False)